# GoogLeNet en base a un algoritmo usado para mnist

Link oritiginal: https://github.com/melty-chocolate/GoogLeNet-Inception-TensorFlow
migrado a TF 2.0

este algoritmo de googlenet no me funciono.
https://github.com/malikmdk/GoogLeNet-with-Tensorflow-2.0

In [1]:
#prueba de gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout,Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
import pydot
import pickle

In [3]:
# se cargan X e y desde los datos preprocesados
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

X = X/255

print( X.shape , y.shape)




(1907, 229, 229, 3) (1907, 32)


In [4]:
def Gnet_model(input_shape, classes):
    
    X_input = Input(input_shape)
  
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (5, 5), name = 'conv1',padding = 'same')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    # MAXPOOL
    X = MaxPooling2D((2, 2),strides = (2, 2), name = 'max_pool1')(X)
    
    # branch1
    # CONV 1X1
    b1_X = Conv2D(64, (1, 1), name = 'conv2',padding = 'valid')(X)
    b1_X = BatchNormalization(axis = 3, name = 'bn2')(b1_X)
    b1_X = Activation('relu')(b1_X)
    b1_X = MaxPooling2D((3, 3), name='max_pool2')(b1_X)

    # branch2
    # CONV 1x1 -> CONV 3x3
    b2_X = Conv2D(64, (1, 1), name = 'conv3',padding = 'same')(X)
    b2_X = BatchNormalization(axis = 3, name = 'bn3')(b2_X)
    b2_X = Activation('relu')(b2_X)
    b2_X = Conv2D(64, (3, 3), name = 'conv4',padding = 'same')(b2_X)
    b2_X = BatchNormalization(axis = 3, name = 'bn4')(b2_X)
    b2_X = Activation('relu')(b2_X)
    b2_X = MaxPooling2D((3, 3), name='max_pool3')(b2_X)
    
    # branch3
    # CONV 1x1 -> CONV 5x5
    b3_X = Conv2D(64, (1, 1), name = 'conv5',padding = 'same')(X)
    b3_X = BatchNormalization(axis = 3, name = 'bn5')(b3_X)
    b3_X = Activation('relu')(b3_X)
    b3_X = Conv2D(64, (5, 5), name = 'conv6',padding = 'same')(b3_X)
    b3_X = BatchNormalization(axis = 3, name = 'bn6')(b3_X)
    b3_X = Activation('relu')(b3_X)
    b3_X = MaxPooling2D((3, 3), name='max_pool4')(b3_X)
    
    # branch4
    # MAXPOOL 3x3 -> CONV 1x1
    b4_X = MaxPooling2D((3, 3), name='max_pool5')(X)
    b4_X = Conv2D(64, (1, 1), name = 'conv7',padding = 'same')(b4_X)
    b4_X = BatchNormalization(axis = 3, name = 'bn7')(b4_X)
    b4_X = Activation('relu')(b4_X)
    
    # CONCAT
    concat = tf.keras.layers.concatenate([b1_X, b2_X, b3_X, b4_X],axis=3)
    X = Activation('relu')(concat)
    
    #X = Flatten()(X)
    X = Dense(128, activation='relu', name='fc1')(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    # Create model. 
    model = Model(inputs = X_input, outputs = X, name='MNIST')

    return model



In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

modelo = Gnet_model(input_shape=(229,229,3), classes = 32)
modelo.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

modelo.summary()

Model: "MNIST"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 229, 229, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 229, 229, 32) 2432        input_1[0][0]                    
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 229, 229, 32) 128         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 229, 229, 32) 0           bn1[0][0]                        
______________________________________________________________________________________________

In [6]:
modelo.fit(X, y, epochs = 20, batch_size = 64 ,validation_split =0.1)

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,64,114,114] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MNIST/bn5/FusedBatchNormV3 (defined at <ipython-input-6-45248ae2f61c>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_2886]

Function call stack:
train_function
